In [13]:
import os
import numpy as np
import pandas as pd
from random import random

from agroplot import GoogleMapPlotter

import webbrowser


In [2]:
# Rutas
ROOT_PATH = 'C:/Users/pedro/ds-geo'
CONFIG_PATH = ROOT_PATH + '/config'
MAPS_PATH = ROOT_PATH + '/maps'
DATA_PATH = ROOT_PATH + '/data'

os.chdir(ROOT_PATH)


# GoogleMaps

In [3]:
class CustomGoogleMapPlotter(GoogleMapPlotter):
    
    # __init__
    def __init__(self, center_lat, center_lng, zoom, apikey='',
                 map_type='satellite'):
        if apikey == '':
            try:
                with open(CONFIG_PATH + '/apikey.txt', 'r') as apifile:
                    apikey = apifile.readline()
            except FileNotFoundError:
                pass
        super().__init__(center_lat, center_lng, zoom, apikey=apikey)

        self.map_type = map_type
        assert(self.map_type in ['roadmap', 'satellite', 'hybrid', 'terrain'])
        

    # write_map
    def write_map(self,  f):
        f.write('\t\tvar centerlatlng = new google.maps.LatLng(%f, %f);\n' %
                (self.center[0], self.center[1]))
        f.write('\t\tvar myOptions = {\n')
        f.write('\t\t\tzoom: %d,\n' % (self.zoom))
        f.write('\t\t\tcenter: centerlatlng,\n')

        # Change this line to allow different map types
        f.write('\t\t\tmapTypeId: \'{}\'\n'.format(self.map_type))
        f.write('\t\t};\n')
        f.write('\t\tvar map = new google.maps.Map(document.getElementById("map_canvas"), myOptions);\n')
        f.write('\n')


    # Añade polígonos y parcelas al mapa
    def AddMapPolygons(self, df, mode=None):

        for index, row in df.iterrows():
            l_lat = row['Latitude']
            l_long = row['Longitude']

            if mode == 'c':
                s_color = row['ColorCultivo']
                if s_color == '':
                    s_color = 'white'
            elif mode == 'p':
                s_color = row['ColorPropietario']
                if s_color == '':
                    s_color = 'white'
            else:
                s_color = 'cornflowerblue'
            
            dict_info = {}
            dict_info.update({'descripcion': row['NombreParcela']})
            dict_info.update({'tipo': row['NombreCultivo']})
            dict_info.update({'tipo_color': row['ColorCultivo']})
            dict_info.update({'variedad': row['VariedadCultivo']})
            dict_info.update({'municipio': row['CatMunicipio']})
            dict_info.update({'poligono': str(row['CatPoligono'])})
            dict_info.update({'parcela': str(row['CatParcela'])})
            dict_info.update({'recinto': str(row['CatRecinto'])})
            dict_info.update({'propietario': row['NombrePropietario']})
            dict_info.update({'propietario_color': row['ColorPropietario']})
           

            self.polygon(l_lat, l_long, info=dict_info, color=s_color, face_alpha=0.5) 

        
        
    # Añade puntos al mapa
    def AddMapPoints(self, df, mode=None):

        for index, row in df.iterrows():
            l_lat = row['Latitude']
            l_long = row['Longitude']

            if mode == 'c':
                s_color = row['ColorCultivo']
                if s_color == '':
                    s_color = 'white'
            elif mode == 'p':
                s_color = 'white'
            else:
                s_color = 'cornflowerblue'
            
            dict_info = {}
            dict_info.update({'descripcion': row['Etiqueta']})
            dict_info.update({'tipo': row['NombreCultivo']})
            dict_info.update({'tipo_color': row['ColorCultivo']})

            self.circle(l_lat, l_long, radius=10, info=dict_info, color=s_color, alpha=0.6)


    # Añade marcadores
    def AddMapMarkers(self, df):
        h = ""
        h = h + "<div id='content'>"
        h = h + "<h2 id='firstHeading' class='firstHeading'>Uluru</h2>"
        h = h + "<div id='bodyContent>"
        h = h + "<p><b>Uluru</b></p>"
        h = h + "</div></div>"

        for index, row in df.iterrows():
            l_lat = row['Latitude']
            l_long = row['Longitude'] 
            s_color = row['ColorCultivo']
            if s_color == '':
                s_color = 'white'
            s_title = row['Etiqueta']
           
            dict_info = {}
            dict_info.update({'descripcion': row['Etiqueta']})
            dict_info.update({'tipo': row['NombreCultivo']})
            dict_info.update({'tipo_color': row['ColorCultivo']})
            
            self.marker(l_lat, l_long, title=s_title, info=dict_info, color=s_color) 

   
    

In [4]:
DATASET_PARCELAS = 'dataset_parcelas.pkl'
DATASET_PUNTOS = 'dataset_puntos.pkl'
DATASET_MARKERS = 'dataset_markers.pkl'

# Función que carga el dataset de las parcelas
def LoadDatasetParcelas():
    df = pd.read_pickle(DATA_PATH + '/' + DATASET_PARCELAS)
    return df

# Función que carga el dataset de los puntos de interés
def LoadDatasetPuntos():
    df = pd.read_pickle(DATA_PATH + '/' + DATASET_PUNTOS)
    return df

# Función que carga el dataset de los markers
def LoadDatasetMarkers():
    df = pd.read_pickle(DATA_PATH + '/' + DATASET_MARKERS)
    return df


In [5]:
# Cálculo del centro del mapa y del zoom inicial (para los ficheros de parcelas KML)

def GetMapInitialSizeKML(df):
    
    max_lat = df['Latitude'].apply(max).max()
    min_lat = df['Latitude'].apply(min).min()
    max_long = df['Longitude'].apply(max).max()
    min_long = df['Longitude'].apply(min).min()
    
    center_lat = min_lat + (max_lat - min_lat)/2
    center_long = min_long + (max_long - min_long)/2
    
    width_lat = abs(max_lat - min_lat)
    width_long = abs(max_long - min_long)
    
    zoom = np.ceil(1.3 * (1 / max(width_lat, width_long)))
    zoom = max(zoom, 12)
    zoom = min(zoom, 17)
    
    return (center_lat, center_long, zoom)



In [6]:
# Generación del mapa con los elementos de la entrada
# Se apoya en la clase CustomGoogleMapPlotter, y a su vez en GoogleMapPlotter
# mode: 'C' = cultivo, 'P' = Propietario
# initial: '' = posición y zoom por defecto, '' = cálculo automático

def GenerateMapComp(polygons, points=None, markers=None, mode='', initial=''):

    if initial == 'auto':
        initial_lat, initial_long, initial_zoom = GetMapInitialSizeKML(polygons)
        print(initial_lat, initial_long, initial_zoom)
    else:
        initial_lat = 39.720578 
        initial_long = -2.924675 
        initial_zoom = 13
    
    s_mode = mode.lower()
    
    gmap = CustomGoogleMapPlotter(initial_lat, initial_long, initial_zoom, map_type='satellite')
    
    if polygons is not None:
        gmap.AddMapPolygons(polygons, s_mode)    
        
    if points is not None:
        gmap.AddMapPoints(points, s_mode)
    
    if markers is not None:
        gmap.AddMapMarkers(markers)
    
    if s_mode == 'c':
        s_mapname = 'mapa_cultivos.html'
    elif s_mode == 'p':
        s_mapname = 'mapa_propietarios.html'
    else:
        s_mapname = 'mapa.html'
        
    gmap.draw(MAPS_PATH + '/' + s_mapname)    

    # Abrir mapa en el navegador
    url = "file://" + MAPS_PATH + '/' + s_mapname
    webbrowser.open_new_tab(url)


### Ejecución

In [7]:
# Carga de las parcelas
df_parcelas = LoadDatasetParcelas()

In [8]:
# Carga puntos destacados
df_puntos = LoadDatasetPuntos()

In [9]:
# Carga de los marcadores
df_markers = LoadDatasetMarkers()

In [15]:
# Genera el mapa y lo abre en el navegador
GenerateMapComp(polygons=df_parcelas, points=df_puntos, mode='c')


39.767443847014 -2.98602125601044 12


In [11]:
#GenerateMapComp(df_parcelas, df_points, df_markers)
#GenerateMapComp(polygons = df_parcelas)
#GenerateMapComp(polygons = df_parcelas, markers=df_markers)

In [12]:
#p = df_parcelas[(df_parcelas['Ref']=='16259A037001760000ZX') & (df_parcelas['Sub'] == 'f')]
#GenerateMapComp(polygons = p)